 # INTRO

Na potrzeby tego projektu został stworzony dedykowany serwis API, który dostępny jest pod adresem: https://api-datalab.coderslab.com/api/v2. Dodatkowo udostępniona została dokumentacja, z którą można zapoznać się tutaj: [klik](https://api-datalab.coderslab.com/v2/docs/).

 > Dokumentacja jest czysto techniczna i ma na celu prezentację dostępnych endpointów wraz ze zwracanym typem. W celu przetestowania należy kliknąć przysisk `Authorize`, podać token (dostępny poniżej), a następnie `Try it out!` oraz uzupełnić wymagane pola (parametry requesta).

 Zgodnie z dokumentacją stwierdzamy, że udostępnione zostały nam 4 endpointy:
 - `airport` - dane o lotnisku,
 - `weather` - informacje o zarejestrowaniej pogodzie na lotnisku danego dnia,
 - `aircraft` - dane o samolotach
 - `flights` - dane o wylotach z danego lotniska per dzień.

 W celu pobrania informacji, gdzie wymagany jest paramatr `airportId`, posłużymy się listą z pliku `airports.csv`.

 ### Uwagi
 - Limit wywołań API to 1000/min, zadbaj o nieprzekroczenie tego limitu – w przeciwnym wypadku będzie zwracany błąd 429.

 # Konfiguracja notatnika

##### Importujemy wymagane biblioteki

In [7]:
import pandas as pd
import requests
import time
from dotenv import load_dotenv
import os

##### Definiujemy paramatry połączenia do API

In [9]:
# Ładujemy zmienne środowiskowe z pliku .env
load_dotenv('DB_pass.env')

True

In [11]:
api_token = os.getenv("api_token")
api_airport_path = "https://api-datalab.coderslab.com/api/v2/airport/"
api_airport__weather_path = "https://api-datalab.coderslab.com/api/v2/airportWeather?date="
api_aircraft_path = "https://api-datalab.coderslab.com/api/v2/aircraft"
api_flight_path = "https://api-datalab.coderslab.com/api/v2/flight?"

##### Wczytujemy plik `airports.csv` i dostosowyjemy go do dalszych kroków w celu pobierania z kolejnych endpointów. Lista lotnisk jest dostępna w kolumnie `origin_airport_id`.

In [9]:
# Wczytujemy plik CSV do DataFrame o nazwie 'airports'
file_path = '../data/airports.csv'
airports = pd.read_csv(file_path)
airports.head()

,origin_airport_id
0,10874
1,11233
2,13360
3,15008
4,11638


In [10]:
# Sprawdzenie ilości wartości NULL (NaN) w kolumnie origin_airport_id
null_count = airports['origin_airport_id'].isna().sum()
print(f"Liczba wartości NULL (NaN) w kolumnie 'origin_airport_id': {null_count}")

# Sprawdzenie ilości duplikatów w kolumnie origin_airport_id
duplicate_count = airports['origin_airport_id'].duplicated().sum()
print(f"Liczba duplikatów w kolumnie 'origin_airport_id': {duplicate_count}")

Liczba wartości NULL (NaN) w kolumnie 'origin_airport_id': 0
Liczba duplikatów w kolumnie 'origin_airport_id': 0


 # Pobieranie `Airport`
Po zapoznaniu się z dokumentacją endpointu `airport`, pobieramy dane dot. poszczególnych lotnisk. Wyniki tego kroku zapisujemy do ramki `airport_df`, a następnie do pliku `csv`.

##### Testujemy podłączenie do endpoint'u `airport`

In [13]:
# Testowanie poprawności zapytania na podstawie jednego lotniska
response = requests.get(f'{api_airport_path}11638', headers={'authorization': api_token})

print('Status:', response.status_code)
print('Cała treść odpowiedzi:', response.json())

Status: 200
Cała treść odpowiedzi: {'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}


##### Pobieramy dane z endpoint'u `airport`

In [15]:
# Lista do przechowywania wyników
airports_details_list = []
airport_not_exit = []

# Licznik zapytań
request_count = 0
max_requests = 1000  # Maksymalna liczba zapytań na minutę

# Iteracja przez każdy 'origin_airport_id' w DataFrame 'airports'
for airport_id in airports['origin_airport_id']:
    # Wysyłanie zapytania do API dla każdego 'origin_airport_id'
    response = requests.get(f'{api_airport_path}{airport_id}', headers={'authorization': api_token})
    
    # Zwiększenie licznika zapytań
    request_count += 1
    
    # Sprawdzenie, czy odpowiedź jest poprawna (status 200)
    if response.status_code == 200:
        data = response.json()  # Parsowanie odpowiedzi JSON
        airports_details_list.append(data)    # Dodanie danych do listy wyników
    else:
        airport_not_exit.append(airport_id)

    # Sprawdzenie, czy osiągnięto limit zapytań
    if request_count == max_requests:
        print("Reached 1000 requests, waiting for 60 seconds...")
        time.sleep(60)  # Pauza na 60 sekund
        request_count = 0  # Zresetowanie licznika zapytań

# Tworzenie nowego DataFrame z wyników
airport_df = pd.DataFrame.from_records(airports_details_list)

In [16]:
# Wyświetlanie listy id lotnisk, których ID nie znajduje się w API
print(f"Airports ID not found in API : {airport_not_exit}")

Airports ID not found in API : [10874, 11233, 13360, 15008, 14150, 15323, 14814, 12007, 11337, 15070, 12280, 11641, 13832, 10268, 15041, 12119, 11537, 11092, 10581, 13829, 15389, 12389, 11648, 15023, 11982, 10967, 11525, 14259, 11637, 10466, 10208, 15841, 12898, 13241, 13367, 11481, 14108, 13873, 10157, 10245, 11146, 13277, 11109, 13459, 11775, 16218, 14698, 14252, 13256, 13139, 12250, 11468, 14952, 12402, 14574, 11977, 11867, 11203, 10747, 14905, 12012, 14783, 10431, 10434, 16869, 10408, 11624, 13541, 13422, 14689, 12391, 10868, 14711, 11067, 10562, 11695, 14109, 13970, 11076, 14092, 11122, 11288, 11308, 10754, 12884, 14588, 12915, 11603, 14457, 12206, 11252, 11905, 14120, 11980, 14025, 11150, 14709, 15897, 14006, 12902, 13061, 12016, 14794, 11921, 10731, 14828, 12441, 14802, 13230, 10631, 10141, 13184, 10643, 10333, 12255, 14487, 12129, 10551, 14256, 13377, 10627, 12335, 14633, 11274, 14543, 10158, 12951, 14004, 12544, 10739, 10165, 13964, 14314, 10990, 14812, 10561, 10146, 15991, 15

In [17]:
# Wyświetlanie pierwszych kilku wierszy nowego DataFrame
airport_df.head()

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
0,11638,Fresno Air Terminal,"Fresno, CA","FRESNO YOSEMITE INTERNATIONAL, CA US"
1,13342,General Mitchell Field,"Milwaukee, WI","MILWAUKEE MITCHELL AIRPORT, WI US"
2,13244,Memphis International,"Memphis, TN","MEMPHIS INTERNATIONAL AIRPORT, TN US"
3,15096,Syracuse Hancock International,"Syracuse, NY","SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US"
4,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...


In [18]:
airport_df.shape

(97, 4)

##### Zapisujemy ramkę `airport_df` do pliku `airport_list.csv` w katalogu `data/raw`.

In [20]:
# Ścieżka do zapisu pliku CSV
file_path = '../data/raw/airport_list.csv'

# Zapisujemy ramkę danych do pliku CSV
airport_df.to_csv(file_path, index=False)

 # Pobieranie `Weather`
Po zapoznaniu się z dokumentacją endpointu Weather, pobieramy dane dot. pogody na poszczególnych lotniskach w okresie od 2019-01-01 do 2020-03-31. Wyniki tego kroku zapisujemy do ramki airport_weather_df, a następnie do pliku csv airport_weather.csv. Ze względu na czas, jaki ten krok może się wykonywać, dodano w pętli informację o ukończonym pobieraniu danych z pełnego miesiąca,aby monitorować przebieg wykonywania tego kroku.

In [22]:
# Testowanie poprawności zapytania na podstawie jednego miesiąca
response = requests.get(f'{api_airport__weather_path}2019-01-01', headers={'authorization': api_token})

print('Status:', response.status_code)
# print('Cała treść odpowiedzi:', response.json()) - ze względu na bardzo długą zawartość odpowiedzi, zostało to zakomentowane
# API zwraca w odpowiedzi dane dla wszystkich lotnisk z całego miesiąca, zamiast tylko z jednego dnia, którego dotyczy zapytanie

Status: 200


In [23]:
# Definiujemy zakres dat do pobrania
start_date = '2019-01-01'
end_date = '2020-03-31'

# Tworzymy listę pierwszych dni miesiąca
first_days_of_months = pd.date_range(start=start_date, end=end_date, freq='MS')
first_days_of_months

DatetimeIndex(['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
               '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
               '2020-01-01', '2020-02-01', '2020-03-01'],
              dtype='datetime64[ns]', freq='MS')

In [24]:
# Lista do przechowywania wyników
airports_weather_details_list = []

# Licznik zapytań
request_count = 0
max_requests = 1000  # Maksymalna liczba zapytań na minutę

# Iteracja zapytań dla każdego miesiąca
for date in first_days_of_months:
    # Wysyłanie zapytania do API dla każdego miesiąca
    response = requests.get(f'{api_airport__weather_path}{date}', headers={'authorization': api_token})
    
    # Zwiększenie licznika zapytań - zapytań tu jest mało, ale dodaję jako dodatkowe zabezpieczenie
    request_count += 1
    
    # Sprawdzenie, czy odpowiedź jest poprawna (status 200)
    if response.status_code == 200:
        data = response.json()  # Parsowanie odpowiedzi JSON
        for record in data:
            airports_weather_details_list.append(record)    # Dodanie danych do listy wyników
    else:
        print(f'Nie można pobrać danych dla daty: {str(date)[:7]} - Status code: {response.status_code}')

    # Sprawdzenie, czy osiągnięto limit zapytań
    if request_count == max_requests:
        print("Reached 1000 requests, waiting for 60 seconds...")
        time.sleep(60)  # Pauza na 60 sekund
        request_count = 0  # Zresetowanie licznika zapytań

    # Wyświetlanie informacji, z którego miesiąca dane zostały już pobrane
    print(f'Pobrano dane dla miesiąca: {str(date)[:7]}')

# Tworzenie nowego DataFrame z wyników
airport_weather_df = pd.DataFrame.from_records(airports_weather_details_list)

Pobrano dane dla miesiąca: 2019-01
Pobrano dane dla miesiąca: 2019-02
Pobrano dane dla miesiąca: 2019-03
Pobrano dane dla miesiąca: 2019-04
Pobrano dane dla miesiąca: 2019-05
Pobrano dane dla miesiąca: 2019-06
Pobrano dane dla miesiąca: 2019-07
Pobrano dane dla miesiąca: 2019-08
Pobrano dane dla miesiąca: 2019-09
Pobrano dane dla miesiąca: 2019-10
Pobrano dane dla miesiąca: 2019-11
Pobrano dane dla miesiąca: 2019-12
Pobrano dane dla miesiąca: 2020-01
Pobrano dane dla miesiąca: 2020-02
Pobrano dane dla miesiąca: 2020-03


In [25]:
airport_weather_df.head()

,WT18,STATION,NAME,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,...,PGTM,WT10,WESD,SN32,SX32,PSUN,TSUN,TOBS,WT07,WT11
0,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-01,4.70,0.14,0.0,0.0,64.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-02,4.92,0.57,0.0,0.0,56.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-03,5.37,0.15,0.0,0.0,52.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-04,12.08,1.44,0.0,0.0,56.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-05,13.42,0.00,0.0,0.0,49.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
airport_weather_df.shape

(46226, 33)

##### Zapisujemy ramkę `airport_weather_df` do pliku `airport_weather.csv` w katalogu `data/raw`.

In [28]:
# Ścieżka do zapisu pliku CSV
file_path = '../data/raw/airport_weather.csv'

# Zapisz ramkę danych do pliku CSV
airport_weather_df.to_csv(file_path, index=False)

 # Pobranie `Aircraft`
Po zapoznaniu się z dokumentacją endpointu Aircraft, pobieramy dane dot. pogody na poszczególnych samolotów. Wyniki tego kroku zapisujemy do ramki aircraft_df, a następnie do pliku csv `aircraft.csv`.

In [30]:
# Testowanie poprawności zapytania
response = requests.get(f'{api_aircraft_path}', headers={'authorization': api_token})

print('Status:', response.status_code)
# print('Cała treść odpowiedzi:', response.json()) - ze względu na bardzo długą zawartość odpowiedzi, zostało to zakomentowane
# API zwraca w odpowiedzi dane dla wszystkich samolotów na raz

Status: 200


In [31]:
# Lista do przechowywania wyników
aircrafts_list = []

# Wysyłanie zapytania do API dla każdego miesiąca
response = requests.get(f'{api_aircraft_path}', headers={'authorization': api_token})
 
# Sprawdzenie, czy odpowiedź jest poprawna (status 200)
if response.status_code == 200:
    data = response.json()  # Parsowanie odpowiedzi JSON
    for record in data:
        aircrafts_list.append(record)    # Dodanie danych do listy wyników
else:
    print(f'Nie można pobrać danych - Status code: {response.status_code}')

# Tworzenie nowego DataFrame z wyników
aircraft_df = pd.DataFrame.from_records(aircrafts_list)

In [32]:
aircraft_df.head()

,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0


In [33]:
aircraft_df.shape

(7383, 3)

##### Zapisujemy ramkę `aircraft_df` do pliku `aircraft.csv` w katalogu `data/raw`.

In [35]:
# Ścieżka do zapisu pliku CSV
file_path = '../data/raw/aircraft.csv'

# Zapisujemy ramkę danych do pliku CSV
aircraft_df.to_csv(file_path, index=False)

 # Pobranie `Flight`
 Zapoznaj się z dokumentacją endpointu `flights`, następnie pobierz dane dotyczące ruchu lotniczego. Wyniki zapisz do ramki `flight_df`, a później do pliku `flight.csv`.

 Wskazówki:
 - Zwróć szczególną uwagę na konstrukcję endpointa,
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego, z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`,
 - Data początkowa danych to `2019-01-01`, zaś końcowa to `2020-03-31`, czyli 456 dni,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku,
 - W przypadku, gdy nie ma dostępnych danych dla danego lotniska, API zwraca kod [204](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/204), w ten sposób możesz pominąć lotniska, dla których dane nie są dostępne,
 - Pobranie całości danych zajmuje dłuższą chwilę, zanim włączysz pętle dla wszystkich danych, sprawdź pobieranie danych dla jednego, dwóch lotnisk aby uniknąć frustracji.

In [37]:
# Testowanie poprawności zapytania na podstawie jednego miesiąca
response = requests.get(f'{api_flight_path}airportId=14082&date=2020-03', headers={'authorization': api_token})

print('Status:', response.status_code)
if response.status_code == 204:
    print("No airport")
else:
    print('Cała treść odpowiedzi:', response.json())

Status: 204
No airport


In [38]:
# Definiujemy zakres dat do pobrania
flight_start_date = '2019-01'
flight_end_date = '2020-03'

# Tworzymy listę pierwszych dni miesiąca
flight_months = pd.date_range(start=start_date, end=end_date, freq='MS')
flight_months = flight_months.strftime('%Y-%m')
flight_months

Index(['2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
       '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
       '2020-01', '2020-02', '2020-03'],
      dtype='object')

In [39]:
# Lista do przechowywania wyników
flight_list = []

# Licznik zapytań
request_count = 0
max_requests = 1000  # Maksymalna liczba zapytań na minutę

# Iteracja przez każdy 'origin_airport_id' w DataFrame 'airports'
for airport_id in airports['origin_airport_id']:
    # Iteracja zapytań dla każdego miesiąca
    for date in flight_months:
        # Wysyłanie zapytania do API dla każdego miesiąca dla każdego lotniska
        response = requests.get(f'{api_flight_path}airportId={airport_id}&date={date}', headers={'authorization': api_token})
        
        # Zwiększenie licznika zapytań
        request_count += 1
        if request_count % 100 == 0:
            print(request_count)
        
        # Sprawdzenie, czy odpowiedź jest poprawna (status 200)
        if response.status_code == 200:
            data = response.json()  # Parsowanie odpowiedzi JSON
            for record in data:
                flight_list.append(record)    # Dodanie danych do listy wyników
        elif response.status_code == 204:
            pass
        else:
            print(f'Nie można pobrać danych dla lotniska {airport_id} na miesiąc {date} - Status code: {response.status_code}')
    
        # Sprawdzenie, czy osiągnięto limit zapytań
        if request_count == max_requests:
            print("Reached 1000 requests, waiting for 60 seconds...")
            time.sleep(60)  # Pauza na 60 sekund
            request_count = 0  # Zresetowanie licznika zapytań

# Tworzenie nowego DataFrame z wyników
flight_df = pd.DataFrame.from_records(flight_list)

100
200
300
400
500
600
700
800
900
1000
Reached 1000 requests, waiting for 60 seconds...
100
200
300
400
500
600
700
800
900
1000
Reached 1000 requests, waiting for 60 seconds...
100
200
300
400
500
600
700
800
900
1000
Reached 1000 requests, waiting for 60 seconds...
100
200
300
400
500
600
700
800
900
1000
Reached 1000 requests, waiting for 60 seconds...
100
200
300
400
500
600
700
800
900
1000
Reached 1000 requests, waiting for 60 seconds...
100
200
300
400


In [40]:
flight_df.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_TIME,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,YEAR,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,1,20,7,WN,N204WN,682,10397,11292,605,602.0,...,205,204.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
1,1,20,7,WN,N8682B,2622,10397,11292,2120,2114.0,...,210,205.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
2,1,20,7,WN,N717SA,2939,10397,11292,1800,1807.0,...,210,220.0,1199,5,2019,4.0,0.0,10.0,0.0,3.0
3,1,20,7,WN,N709SW,3848,10397,11292,1355,1354.0,...,205,204.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
4,1,20,7,WN,N7864B,1352,10397,11697,1125,1125.0,...,120,124.0,581,3,2019,NaN,NaN,NaN,NaN,NaN


In [41]:
flight_df.shape

(1386120, 27)

##### Zapisujemy ramkę `flight_df` do pliku `flight.csv` w katalogu `data/raw`.

In [45]:
# Ścieżka do zapisu pliku CSV
file_path = '../data/raw/flight.csv'

# Zapisujemy ramkę danych do pliku CSV
flight_df.to_csv(file_path, index=False)

 # Podsumowanie
W tym notatniku wykonaliśmy podstawowy krok w analizie danych - pozyskaliśmy je. Dane są gotowe do dalszej pracy, czyli możemy załadować je na bazę danych, a następnie zapoznać się z tym, jakie informacje ze sobą niosą.

In [47]:
msg = "Wszystko wygląda OK :) Przechodzimy do kolejnego kroku."
print(msg)

Wszystko wygląda OK :) Przechodzimy do kolejnego kroku.
